In [5]:
import numpy as  np
import cv2

In [ ]:
webcam = cv2.VideoCapture(0,cv2.CAP_DSHOW) 

while True:
    
    ret,current_frame = webcam.read()
    
    height = current_frame.shape[0]
    width = current_frame.shape[1]
    img_blob = cv2.dnn.blobFromImage(current_frame,swapRB=True,crop=False) # convert BGR to RGB without cropping
    
    class_labels = ["person","bicycle","car","motorbike","aeroplane","bus","train","truck","boat","traffic light",
                    "fire hydrant","street sign","stop sign","parking meter","bench","bird","cat","dog","horse",
                    "sheep","cow","elephant","bear","zebra","giraffe","hat","backpack","umbrella","shoe","eye glasses",
                    "handbag","tie","suitcase","frisbee","skis","snowboard","sports ball","kite","baseball bat","baseball glove",
                    "skateboard","surfboard","tennis racket","bottle","plate","wine glass","cup","fork","knife",
                    "spoon","bowl","banana","apple","sandwich","orange","broccoli","carrot","hot dog","pizza","donut",
                    "cake","chair","sofa","pottedplant","bed","mirror","diningtable","window","desk","toilet","door","tv",
                    "laptop","mouse","remote","keyboard","cell phone","microwave","oven","toaster","sink","refrigerator",
                    "blender","book","clock","vase","scissors","teddy bear","hair drier","toothbrush"]
    
    # Red, Lime, Blue, Yellow, Cyan, Magenta, Silver, Gray, Maroon, Olive, Green, Purple, Teal, Navy, random_color
    class_colors = ["255,0,0","0,255,0","0,0,255","255,255,0","0,255,255","255,0,255","192,192,192","128,128,128",
                    "128,0,0","128,128,0","0,128,0","128,0,128","0,128,128","0,0,128","25,191,88"]
    
    class_colors = [np.array(x.split(",")).astype("int") for x in class_colors]
    class_colors = np.array(class_colors)
    class_colors = np.tile(class_colors,(6,1))

    maskrcnn = cv2.dnn.readNetFromTensorflow('maskrcnn_buffermodel.pb','maskrcnn_bufferconfig.txt')
    maskrcnn.setInput(img_blob)
    (boxes, masks)  = maskrcnn.forward(["detection_out_final","detection_masks"])
    
    no_of_detections = boxes.shape[2]
    
    for i in np.arange(0, no_of_detections):
        prediction_confidence = boxes[0, 0, i, 2]
        
        if prediction_confidence >= 0.40:

            predicted_class_index = int(boxes[0, 0, i, 1])
            predicted_class_label = class_labels[predicted_class_index]
            
            bounding_box = boxes[0, 0, i, 3:7] * np.array([width, height, width, height])
            (start_x, start_y, end_x, end_y) = bounding_box.astype("int")
            
            #obtain width and height of bounding box
            bounding_box_Width = (end_x - start_x)
            bounding_box_Height = (end_y - start_y)
            
            object_mask = masks[i, predicted_class_index]
            object_mask = cv2.resize(object_mask, (bounding_box_Width, bounding_box_Height))
        
            object_mask = (object_mask > 0.3)
        
            roi = current_frame[start_y:end_y, start_x:end_x]
            roi = roi[object_mask]
        
            mask_color = class_colors[predicted_class_index]
            
            roi_color_transparent_cover = ((0.3 * mask_color) + (0.5 * roi)).astype("uint8")
            current_frame[start_y:end_y, start_x:end_x][object_mask] = roi_color_transparent_cover
            mask_color = [int(c) for c in mask_color]
            
            predicted_class_label = "{}: {:.2f}%".format(class_labels[predicted_class_index], prediction_confidence * 100)
            

            cv2.rectangle(current_frame, (start_x, start_y), (end_x, end_y), mask_color, 2)
            cv2.putText(current_frame, predicted_class_label, (start_x, start_y -7), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,255,255), 1)
    
    
    cv2.imshow("xyz", current_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

webcam.release()
cv2.destroyAllWindows() 